# Gizli Dirichlet Dağılımı (Latent Dirichlet Allocation) (LDA)

🎯 Bu challenge'ın amacı, **LDA** algoritması (NLP'de Denetimsiz Öğrenme) ile e-posta külliyatı içinde konular bulmaktır.

✉️ İşte 1000'den fazla ***etiketlenmemiş e-posta*** içeren bir koleksiyon. Bunlardan ***konuları çıkarmaya*** çalışalım!

In [1]:
import pandas as pd

url = 'https://d32aokrjazspmn.cloudfront.net/materials/lda_data'

data = pd.read_csv(url, sep=",", header=None)
data.columns = ['text']
data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


In [3]:
data.shape

(1199, 1)

## (1) Preprocessing 

❓ **Question (Cleaning**) ❓ You're used to it by now... Clean up! Store the cleaned text in a new column "clean_text" of the DataFrame.

In [4]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Gerekli NLTK paketlerini kontrol edelim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def clean_data(text):
    # 1. Noktalama işaretlerini kaldır
    text = "".join([char for char in text if char not in string.punctuation])
    
    # 2. Sayıları kaldır
    text = "".join([char for char in text if not char.isdigit()])
    
    # 3. Küçük harfe çevir
    text = text.lower()
    
    # 4. Tokenize (Kelimelere ayır)
    tokens = word_tokenize(text)
    
    # 5. Stopwords ve Lemmatization (Aynı döngüde halledelim)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Kelime stopword değilse -> kökünü bul ve listeye ekle
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # 6. Tekrar birleştir
    return " ".join(clean_tokens)

# Fonksiyonu tüm satırlara uygula
data['clean_text'] = data['text'].apply(clean_data)

# Sonucu kontrol et
data.head()

[nltk_data] Downloading package stopwords to /home/tumay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tumay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tumay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tumay/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text,clean_text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,gldcunixbcccolumbiaedu gary l dare subject sta...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...,atterlepvelaacsoaklandedu cardinal ximenez sub...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...,minerkuhubccukansedu subject ancient book orga...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...,atterlepvelaacsoaklandedu cardinal ximenez sub...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...,vzhivovsuperiorcarletonca vladimir zhivov subj...


## (2) Latent Dirichlet Allocation model

❓ **Soru (Eğitim)** ❓ Potansiyel konuları çıkarmak için bir LDA modeli eğitin

Konu Anlatımı: Temizlenmiş metinlerimiz var ama bilgisayar hala bunları kelime olarak görüyor. LDA modelini çalıştırmadan önce yapmamız gereken iki kritik hamle var:

Vektörleştirme (Yine!): Ham/Spam ödevindeki gibi CountVectorizer kullanacağız. Metinleri sayısal bir matrise çevireceğiz. LDA, genellikle kelime sıklıklarıyla (Bag-of-Words) çalışmayı sever.

Konu Sayısını Seçme (n_components): Bu en kritik kısımdır. Denetimsiz öğrenme olduğu için bilgisayar kaç konu olduğunu bilmez. Biz ona bir ipucu vermeliyiz: "Bana bu metinleri 2 ana başlık altında topla" veya "Bana 3 konu bul" demeliyiz.

Başlangıç için 2 konuyu deneyelim. Bakalım e-postaları güzelce ikiye ayırabilecek mi?

In [5]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 1. Vektörleştiriciyi (Sayma Makinesi) Hazırla
# Bu sefer max_df ve min_df parametrelerini kullanmıyoruz ama 
# gerçek projelerde çok sık veya çok nadir geçen kelimeleri filtrelemek iyidir.
vectorizer = CountVectorizer()

# Metni sayısal matrise çevir
data_vectorized = vectorizer.fit_transform(data['clean_text'])

# 2. LDA Modelini Oluştur
# n_components=2 -> "Bana bu verinin içinden 2 tane ana konu bul" diyoruz.
lda_model = LatentDirichletAllocation(n_components=2, random_state=42)

# 3. Modeli Eğit
lda_model.fit(data_vectorized)

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",2
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",10.0
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


##  (3) Potansiyel konuları görselleştirin

🎁 Potansiyel konularla ilişkili kelimeleri yazdırmak için bir  fonksiyon kodladık.

In [6]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

❓ **Soru** ❓ LDA tarafından çıkarılan konuları yazdırın.

In [8]:
# Fonksiyonu çağırıyoruz:
# 1. lda_model: Eğittiğimiz model
# 2. vectorizer: Kelime sözlüğümüz
print_topics(lda_model, vectorizer)

Topic 0:
[('team', np.float64(954.492583614096)), ('game', np.float64(926.6763857533701)), ('line', np.float64(705.8911900292647)), ('subject', np.float64(631.5763931401136)), ('organization', np.float64(609.7109699597447)), ('hockey', np.float64(594.4924889314436)), ('player', np.float64(524.466852174314)), ('play', np.float64(495.7560584698419)), ('year', np.float64(443.086532643443)), ('university', np.float64(421.04582301590835))]
Topic 1:
[('god', np.float64(1490.4722687452693)), ('one', np.float64(838.1900693896296)), ('would', np.float64(809.9103911151723)), ('christian', np.float64(758.4705977219897)), ('people', np.float64(700.3520982523543)), ('subject', np.float64(675.4236068598353)), ('line', np.float64(638.1088099706847)), ('jesus', np.float64(623.4900913689678)), ('organization', np.float64(561.2890300402075)), ('say', np.float64(540.5486730510962))]


## (4) Yeni bir metnin belge-konu karışımını tahmin edin

❓ **Soru (Tahmin)** ❓

LDA modeliniz fit edildiğine göre, onu yeni bir metnin konularını tahmin etmek için kullanabilirsiniz.

1. Örneği vektörleştirin
2. Vektörleştirilmiş örnek üzerinde LDA'yı kullanarak konuları tahmin edin

In [9]:
example = ["My team performed poorly last season. Their best player was out injured and only played one game"]

In [10]:
# Örnek cümlemiz (Not defterinde zaten tanımlı ama tekrar hatırlayalım)
example = ["My team performed poorly last season. Their best player was out injured and only played one game"]

# 1. Örneği Vektörleştir (Sadece transform!)
# "Bu cümleyi, bildiğin kelime havuzuna göre sayıya çevir" diyoruz.
example_vectorized = vectorizer.transform(example)

# 2. LDA Modeli ile Tahmin Et
# "Bu sayısal verinin hangi konuya ne kadar yakın olduğunu hesapla"
lda_vectors = lda_model.transform(example_vectorized)

# Sonucu yazdıralım
print("Tahmin Sonuçları:", lda_vectors)

# Daha anlaşılır görmek için:
print(f"Konu 0 (Spor) Olasılığı: %{lda_vectors[0][0] * 100:.2f}")
print(f"Konu 1 (Din) Olasılığı:  %{lda_vectors[0][1] * 100:.2f}")

Tahmin Sonuçları: [[0.95167532 0.04832468]]
Konu 0 (Spor) Olasılığı: %95.17
Konu 1 (Din) Olasılığı:  %4.83


🏁 Tebrikler! LDA'yı hızlı bir şekilde nasıl uygulayacağınızı öğrendiniz.

💾 Not defterinizi `git add/commit/push` yapmayı unutmayın...

🚀 ... ve bir sonraki göreve geçin!